In [1]:
# Import Library yang dibutuhkan
import time
import numpy as np
import imclab 
import paho.mqtt.client as mqtt

print("Semua library siap.")

Semua library siap.


In [2]:
# --- KONFIGURASI MQTT ---
# Gunakan broker publik gratis
MQTT_BROKER = "broker.hivemq.com" 
MQTT_PORT = 1883
# GANTI DENGAN NAMA PROYEK/USERNAME UNIK ANDA!
MQTT_TOPIC_ROOT = "iot_imclab" 

# Topik untuk SUBSCRIBE (Menerima Perintah dari HP)
TOPIC_SETPOINT = f"{MQTT_TOPIC_ROOT}/setpoint"
TOPIC_KP = f"{MQTT_TOPIC_ROOT}/kp"
TOPIC_KI = f"{MQTT_TOPIC_ROOT}/ki"
TOPIC_KD = f"{MQTT_TOPIC_ROOT}/kd"

# Topik untuk PUBLISH (Mengirim Data ke HP)
TOPIC_RPM = f"{MQTT_TOPIC_ROOT}/rpm"
TOPIC_OUTPUT = f"{MQTT_TOPIC_ROOT}/output"
TOPIC_ERROR = f"{MQTT_TOPIC_ROOT}/error"

# --- VARIABEL PID GLOBAL ---
# Nilai awal yang bisa diubah via HP
setpoint = 0.0
kp = 0.5   
ki = 0.1
kd = 0.005 # Gunakan nilai D kecil dulu
prev_error = 0.0
integral = 0.0
last_time = 0

# Variabel untuk Data Logging
log_data = []

print(f"MQTT Root Topic: {MQTT_TOPIC_ROOT}")

MQTT Root Topic: iot_imclab


In [3]:
# --- FUNGSI CALLBACK MQTT ---
def on_connect(client, userdata, flags, rc):
    """Dipanggil ketika klien menerima respons CONNECT dari broker."""
    if rc == 0:
        print(f"✅ Terhubung ke MQTT Broker ({MQTT_BROKER})")
        # Subscribe ke semua topik kontrol
        client.subscribe([(TOPIC_SETPOINT, 0), (TOPIC_KP, 0), 
                          (TOPIC_KI, 0), (TOPIC_KD, 0)])
    else:
        print(f"❌ Gagal Terhubung, kode: {rc}")

def on_message(client, userdata, msg):
    """Dipanggil ketika pesan SUBSCRIBE diterima dari broker."""
    global setpoint, kp, ki, kd, integral
    try:
        val = float(msg.payload.decode())
        topic = msg.topic
        
        if topic == TOPIC_SETPOINT:
            if setpoint != val:
                setpoint = val
                integral = 0.0 
                print(f"Setpoint Diperbarui: {setpoint} RPM")
        elif topic == TOPIC_KP:
            kp = val
            print(f"Koefisien Kp Diperbarui: {kp}")
        elif topic == TOPIC_KI:
            ki = val
            print(f"Koefisien Ki Diperbarui: {ki}")
        elif topic == TOPIC_KD:
            kd = val
            print(f"Koefisien Kd Diperbarui: {kd}")
            
    except ValueError:
        pass # Abaikan error konversi agar tidak mengganggu log

# --- INISIALISASI KONEKSI ---
# Pastikan mematikan client lama sebelum membuat yang baru jika restart kernel
try:
    client.loop_stop()
except:
    pass

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

print("Mencoba koneksi ke MQTT...")
client.connect(MQTT_BROKER, MQTT_PORT, 60)
client.loop_start()

Mencoba koneksi ke MQTT...


/var/folders/11/hzb5vsm901q_v13vx1gfzh4m0000gn/T/ipykernel_40171/741554824.py:44: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [4]:
# --- KONEKSI IMCLAB ---
try:
    lab = imclab.iMCLab()
    print("✅ iMCLab/ESP32 Berhasil Terhubung!")
    time.sleep(1)
except Exception as e:
    print(f"❌ GAGAL: {e}")
    # Jika gagal, pastikan port dan driver sudah benar
    # Jika perlu, coba hardcode port seperti yang dijelaskan sebelumnya
    lab = None

Opening connection
✅ Terhubung ke MQTT Broker (broker.hivemq.com)
iMCLab connected via Arduino on port /dev/cu.usbserial-0001
✅ iMCLab/ESP32 Berhasil Terhubung!


In [7]:
# --- MAIN PID LOOP (FINAL FIX: Setpoint Line) ---
global prev_error, integral, last_time, log_data

# Setting Kecepatan Update ke HP (Detik)
MQTT_UPDATE_INTERVAL = 1

if lab is None:
    print("⚠️ Koneksi iMCLab tidak ditemukan.")
else:
    print(f"--- PID Loop Dimulai (Update HP setiap {MQTT_UPDATE_INTERVAL}s) ---")
    
    start_time = time.time()
    last_time = start_time
    last_mqtt_time = start_time 
    
    try:
        while True:
            current_time = time.time()
            dt = current_time - last_time
            
            # --- 1. LOOP PID (CEPAT: 10Hz) ---
            if dt >= 0.1: 
                
                # Baca RPM
                current_rpm = lab.RPM 
                
                # Hitung PID
                error = setpoint - current_rpm
                
                P = kp * error
                
                integral += error * dt
                integral = max(-100, min(100, integral)) 
                I = ki * integral
                
                D = kd * (error - prev_error) / dt if dt > 0 else 0
                
                output_signal = P + I + D
                output_signal = max(0.0, min(100.0, output_signal))
                
                # Kirim ke Motor
                lab.op(output_signal)
                
                # Update variabel
                prev_error = error
                last_time = current_time
                
                # Logging Data
                log_data.append([current_time - start_time, output_signal, current_rpm, setpoint])

                # --- 2. LOOP MQTT (LAMBAT: 0.5 Detik) ---
                if (current_time - last_mqtt_time) > MQTT_UPDATE_INTERVAL:
                    
                    client.publish(TOPIC_SETPOINT, setpoint) 
                    client.publish(TOPIC_RPM, round(current_rpm, 0))
                    client.publish(TOPIC_OUTPUT, round(output_signal, 1))
                    client.publish(TOPIC_ERROR, round(error, 1))
                    
                    last_mqtt_time = current_time
                    
                    print(f"T: {current_time-start_time:.1f} | SP: {setpoint:.0f} | RPM: {current_rpm:.0f} | Out: {output_signal:.1f}%")

            time.sleep(0.005) 
            
    except KeyboardInterrupt:
        print("\nPID Loop Dihentikan.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
        
    finally:
        lab.op(0)
        print("Motor dimatikan.")

--- PID Loop Dimulai (Update HP setiap 1s) ---
T: 1.0 | SP: 1 | RPM: 1075 | Out: 0.0%
T: 2.1 | SP: 1 | RPM: 231 | Out: 0.0%
T: 3.1 | SP: 1 | RPM: 261 | Out: 0.0%
T: 4.2 | SP: 1 | RPM: 0 | Out: 0.0%
T: 5.2 | SP: 1 | RPM: 288 | Out: 0.0%
T: 6.2 | SP: 1 | RPM: 0 | Out: 0.0%
Setpoint Diperbarui: 3000.7 RPM
Setpoint Diperbarui: 1.0 RPM
T: 7.3 | SP: 1 | RPM: 203 | Out: 0.0%
T: 8.3 | SP: 1 | RPM: 0 | Out: 0.0%
T: 9.4 | SP: 1 | RPM: 289 | Out: 0.0%
T: 10.4 | SP: 1 | RPM: 0 | Out: 0.0%
T: 11.4 | SP: 1 | RPM: 233 | Out: 0.0%
T: 12.5 | SP: 1 | RPM: 0 | Out: 0.0%
Setpoint Diperbarui: 4300.57 RPM
T: 13.5 | SP: 4301 | RPM: 231 | Out: 100.0%
Setpoint Diperbarui: 3000.7 RPM
T: 14.6 | SP: 3001 | RPM: 4753 | Out: 0.0%
T: 15.6 | SP: 3001 | RPM: 3545 | Out: 0.0%
T: 16.6 | SP: 3001 | RPM: 232 | Out: 100.0%
T: 17.7 | SP: 3001 | RPM: 6792 | Out: 0.0%
T: 18.7 | SP: 3001 | RPM: 4109 | Out: 0.0%
T: 19.8 | SP: 3001 | RPM: 261 | Out: 100.0%
T: 20.8 | SP: 3001 | RPM: 6819 | Out: 0.0%
T: 21.8 | SP: 3001 | RPM: 4146